# 02 – Feature Engineering and Analytic Dataset for Telco Churn

This notebook builds the **cleaned analytic dataset** for the IBM Telco Customer Churn project.

Goals:

- Clean key numeric fields.
- Handle missing values relevant for BI analysis.
- Create engineered features used in KPIs and Tableau dashboards.
- Export a single, consistent CSV file for downstream visualization.


In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


## Load Raw Dataset

We load the original IBM Telco Customer Churn Excel file from `data/raw/`.


In [2]:
file_path = "../data/raw/Telco_customer_churn.xlsx"

df = pd.read_excel(file_path)
df.head()


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.96,-118.27,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.06,-118.31,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.05,-118.29,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.06,-118.32,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,"3,046.05",Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.04,-118.27,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,"5,036.30",Yes,1,89,5340,Competitor had better devices


## Initial Shape and Structure

We quickly re-check the shape and structure to confirm it matches the understanding from the first notebook.


In [3]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

df.info()


Rows: 7043
Columns: 33
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  I

## Clean Numeric Fields

The following columns must be treated as numeric for analysis and KPIs:

- `Monthly Charges`
- `Total Charges`
- `CLTV`
- `Churn Value` (numeric target)

In the raw file, some of them may be stored as text.  
We convert them to numeric types, coercing invalid strings to `NaN` (to be handled in the next step).


In [4]:
numeric_cols = ['Monthly Charges', 'Total Charges', 'CLTV']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Ensure churn value is integer (0/1)
df['ChurnValue'] = df['Churn Value'].astype(int)

df[numeric_cols + ['ChurnValue']].describe()


,Monthly Charges,Total Charges,CLTV,ChurnValue
count,"7,043.00","7,032.00","7,043.00","7,043.00"
mean,64.76,"2,283.30","4,400.30",0.27
std,30.09,"2,266.77","1,183.06",0.44
min,18.25,18.80,"2,003.00",0.00
25%,35.50,401.45,"3,469.00",0.00
50%,70.35,"1,397.47","4,527.00",0.00
75%,89.85,"3,794.74","5,380.50",1.00
max,118.75,"8,684.80","6,500.00",1.00


## Handle Missing Values

There are two main sources of missing values relevant for this analytic dataset:

1. **`Total Charges`**  
   - New customers can have blank or non-parsable `Total Charges` values.  
   - For BI dashboards, we replace missing `Total Charges` with `0.0`, which is acceptable for very new customers.

2. **`Churn Reason`**  
   - This field is only defined for churned customers.  
   - For active (non-churned) customers it is missing.  
   - To avoid `NULL` categories in Tableau, we replace missing values with a clear label:
     `"No churn / active customer"`.


In [5]:
# Total Charges: replace missing with 0 for new customers
df['Total Charges'] = df['Total Charges'].fillna(0)

# Churn Reason: label missing as "No churn / active customer"
df['Churn Reason'] = df['Churn Reason'].fillna("No churn / active customer")

# Quick check of remaining missing values
df.isna().sum().sort_values(ascending=False).head(10)


CustomerID    0
Count         0
Country       0
State         0
City          0
Zip Code      0
Lat Long      0
Latitude      0
Longitude     0
Gender        0
dtype: int64

## Feature Engineering

We now create the engineered fields that will be used in KPIs and Tableau dashboards:

1. **`ChurnFlag`** – binary 0/1 flag based on `Churn Label`.
2. **`TenureGroup`** – tenure cohorts (0–6, 7–12, 13–24, 25–48, 49+ months).
3. **`CLTV_Segment`** – customer lifetime value segments (Low / Medium / High).
4. **`RevenueAtRisk`** – monthly revenue at risk from churned customers.
5. **`RiskBucket`** – churn risk buckets based on `Churn Score` (Low / Medium / High).


In [6]:
# 1. ChurnFlag
df['ChurnFlag'] = (df['Churn Label'] == 'Yes').astype(int)

df['ChurnFlag'].value_counts()


ChurnFlag
0    5174
1    1869
Name: count, dtype: int64

In [7]:
# 2. TenureGroup
def tenure_group(x):
    if x <= 6:
        return "0-6"
    elif x <= 12:
        return "7-12"
    elif x <= 24:
        return "13-24"
    elif x <= 48:
        return "25-48"
    else:
        return "49+"

df['TenureGroup'] = df['Tenure Months'].apply(tenure_group)

df['TenureGroup'].value_counts().sort_index()


TenureGroup
0-6      1481
13-24    1024
25-48    1594
49+      2239
7-12      705
Name: count, dtype: int64

In [8]:
# 3. CLTV_Segment
def cltv_segment(x):
    if x >= 5000:
        return "High"
    elif x >= 2500:
        return "Medium"
    else:
        return "Low"

df['CLTV_Segment'] = df['CLTV'].apply(cltv_segment)

df['CLTV_Segment'].value_counts()


CLTV_Segment
Medium    3897
High      2574
Low        572
Name: count, dtype: int64

In [9]:
# 4. RevenueAtRisk
df['RevenueAtRisk'] = df['Monthly Charges'] * df['ChurnFlag']

df['RevenueAtRisk'].describe()


count   7,043.00
mean       19.75
std        35.24
min         0.00
25%         0.00
50%         0.00
75%        24.10
max       118.35
Name: RevenueAtRisk, dtype: float64

In [10]:
# 5. RiskBucket based on Churn Score
def risk_bucket(score):
    if score >= 70:
        return "High Risk"
    elif score >= 40:
        return "Medium Risk"
    else:
        return "Low Risk"

df['RiskBucket'] = df['Churn Score'].apply(risk_bucket)

df['RiskBucket'].value_counts()


RiskBucket
Medium Risk    2805
High Risk      2559
Low Risk       1679
Name: count, dtype: int64

## Quick Sanity Checks on Engineered Features

We verify that:

- `ChurnFlag` matches `Churn Label` (Yes → 1, No → 0).
- `TenureGroup` covers all records with the expected bins.
- `CLTV_Segment` has three levels: Low, Medium, High.
- `RiskBucket` has three levels: Low Risk, Medium Risk, High Risk.
- No new missing values were introduced by the feature engineering steps.


In [11]:
print("ChurnFlag vs Churn Label crosstab:")
print(pd.crosstab(df['Churn Label'], df['ChurnFlag']))

print("\nTenureGroup distribution:")
print(df['TenureGroup'].value_counts().sort_index())

print("\nCLTV_Segment distribution:")
print(df['CLTV_Segment'].value_counts())

print("\nRiskBucket distribution:")
print(df['RiskBucket'].value_counts())

print("\nTotal missing values in the dataset:", df.isna().sum().sum())


ChurnFlag vs Churn Label crosstab:
ChurnFlag       0     1
Churn Label            
No           5174     0
Yes             0  1869

TenureGroup distribution:
TenureGroup
0-6      1481
13-24    1024
25-48    1594
49+      2239
7-12      705
Name: count, dtype: int64

CLTV_Segment distribution:
CLTV_Segment
Medium    3897
High      2574
Low        572
Name: count, dtype: int64

RiskBucket distribution:
RiskBucket
Medium Risk    2805
High Risk      2559
Low Risk       1679
Name: count, dtype: int64

Total missing values in the dataset: 0


## Export Analytic Dataset for Tableau

We now export the cleaned and feature-engineered dataset to:

- `data/processed/telco_churn_clean.csv`

This CSV will be the **single source of truth** for Tableau dashboards and KPI calculations.


In [12]:
output_path = "../data/processed/telco_churn_clean.csv"
df.to_csv(output_path, index=False)

output_path


'../data/processed/telco_churn_clean.csv'

## Final Validation

As a final step, we re-check:

- Row and column counts.
- Total number of missing values (should be 0 after handling `Churn Reason` and numeric fields).


In [13]:
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

total_missing = df.isna().sum().sum()
print("Total missing values:", total_missing)

if total_missing == 0:
    print("\n✅ Dataset is fully populated (no missing values).")
else:
    print("\n⚠ There are still missing values – please investigate.")


Rows: 7043
Columns: 39
Total missing values: 0

✅ Dataset is fully populated (no missing values).


## Summary and Next Steps

In this notebook we:

- Cleaned key numeric fields (`Monthly Charges`, `Total Charges`, `CLTV`, `Churn Value`).
- Handled missing values:
  - Replaced missing `Total Charges` with `0.0` for very new customers.
  - Replaced missing `Churn Reason` with `"No churn / active customer"` for non-churned records.
- Engineered features for analysis and dashboards:
  - `ChurnFlag` (0/1)
  - `TenureGroup`
  - `CLTV_Segment`
  - `RevenueAtRisk`
  - `RiskBucket` (based on `Churn Score`)
- Exported the analytic dataset to `data/processed/telco_churn_clean.csv`.
- Confirmed that the final dataset contains no missing values.

**Next steps (next PR / notebook):**

- Define KPIs using these engineered fields.
- Connect Tableau to `telco_churn_clean.csv`.
- Build dashboards for:
  - Executive overview
  - Churn drivers & segmentation
  - Geo/demographic profile
  - Risk & retention recommendations
